In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "COVID-19"
cohort = "GSE273225"

# Input paths
in_trait_dir = "../../input/GEO/COVID-19"
in_cohort_dir = "../../input/GEO/COVID-19/GSE273225"

# Output paths
out_data_file = "../../output/preprocess/COVID-19/GSE273225.csv"
out_gene_data_file = "../../output/preprocess/COVID-19/gene_data/GSE273225.csv"
out_clinical_data_file = "../../output/preprocess/COVID-19/clinical_data/GSE273225.csv"
json_path = "../../output/preprocess/COVID-19/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"The effect of rewarming ischemia on tissue transcriptome signatures: a clinical observational study in lung transplantation"
!Series_summary	"BACKGROUND: In lung transplantation (LuTx), various ischemic phases exist, yet the rewarming ischemia time (RIT) during implantation has often been overlooked. During RIT, lungs are deflated and exposed to the body temperature in the recipient's chest cavity. Our prior clinical findings demonstrated that prolonged RIT increases the risk of primary graft dysfunction. However, the molecular mechanisms of rewarming ischemic injury in this context remain unexplored. We aimed to characterize the rewarming ischemia phase during LuTx by measuring organ temperature and comparing transcriptome and metabolome profiles in tissue obtained at the end versus the start of implantation."
!Series_summary	"METHODS: In a clinical observational study, 34 double-LuTx with ice preservation were analyzed. Lung core and surface tem

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the Series_overall_design and the overall structure, this study includes
# gene expression data using nCounter digital gene expression analysis with the Immunology V2 panel
# This targets 579 immune system-associated genes, which is suitable for our analysis
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Trait related data - Check for COVID-19 related data
# Looking at the background info, this is a lung transplantation study with no indication of COVID-19
# The study is about rewarming ischemia time during lung transplantation
trait_row = None  # No COVID-19 data available

# 2.2 Age data
# Available in key 3 as "donor age (y): XX"
age_row = 3

def convert_age(value):
    """Convert age value to numeric (continuous)"""
    if value is None:
        return None
    try:
        # Extract the number after the colon
        if ":" in value:
            age_str = value.split(":")[1].strip()
            return float(age_str)
        return None
    except:
        return None

# 2.3 Gender data
# Available in key 4 as "donor sex: male/female"
gender_row = 4

def convert_gender(value):
    """Convert gender information to binary (0 for female, 1 for male)"""
    if value is None:
        return None
    try:
        # Extract the gender after the colon
        if ":" in value:
            gender = value.split(":")[1].strip().lower()
            if gender == "female":
                return 0
            elif gender == "male":
                return 1
        return None
    except:
        return None

# For completeness, define convert_trait even though we don't have COVID-19 data
def convert_trait(value):
    """
    Convert trait information to binary.
    This is a placeholder as there's no COVID-19 data in this dataset.
    """
    return None

# 3. Save Metadata
# Determine if trait data is available (it's not)
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step as trait_row is None (trait data not available)


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
marker_row = None
try:
    with gzip.open(matrix_file, 'rt') as file:
        for i, line in enumerate(file):
            if "!series_matrix_table_begin" in line:
                found_marker = True
                marker_row = i
                print(f"Found the matrix table marker at line {i}")
                break
    
    if not found_marker:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        is_gene_available = False
        
    # If marker was found, try to extract gene data
    if is_gene_available:
        try:
            # Try using the library function
            gene_data = get_genetic_data(matrix_file)
            
            if gene_data.shape[0] == 0:
                print("Warning: Extracted gene data has 0 rows.")
                is_gene_available = False
            else:
                print(f"Gene data shape: {gene_data.shape}")
                # Print the first 20 gene/probe identifiers
                print("First 20 gene/probe identifiers:")
                print(gene_data.index[:20].tolist())
        except Exception as e:
            print(f"Error extracting gene data with get_genetic_data(): {e}")
            is_gene_available = False
    
    # If gene data extraction failed, examine file content to diagnose
    if not is_gene_available:
        print("Examining file content to diagnose the issue:")
        try:
            with gzip.open(matrix_file, 'rt') as file:
                # Print lines around the marker if found
                if marker_row is not None:
                    for i, line in enumerate(file):
                        if i >= marker_row - 2 and i <= marker_row + 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        if i > marker_row + 10:
                            break
                else:
                    # If marker not found, print first 10 lines
                    for i, line in enumerate(file):
                        if i < 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        else:
                            break
        except Exception as e2:
            print(f"Error examining file: {e2}")
        
except Exception as e:
    print(f"Error processing file: {e}")
    is_gene_available = False

# Update validation information if gene data extraction failed
if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")
    # Update the validation record since gene data isn't available
    is_trait_available = False  # We already determined trait data isn't available in step 2
    validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path,
                                 is_gene_available=is_gene_available, is_trait_available=is_trait_available)


SOFT file: ../../input/GEO/COVID-19/GSE273225/GSE273225_family.soft.gz
Matrix file: ../../input/GEO/COVID-19/GSE273225/GSE273225_series_matrix.txt.gz
Found the matrix table marker at line 80
Gene data shape: (608, 118)
First 20 gene/probe identifiers:
['ABCB1', 'ABCF1', 'ABL1', 'ADA', 'AHR', 'AICDA', 'AIRE', 'ALAS1', 'APP', 'ARG1', 'ARG2', 'ARHGDIB', 'ATG10', 'ATG12', 'ATG16L1', 'ATG5', 'ATG7', 'ATM', 'B2M', 'B3GAT1']


### Step 4: Gene Identifier Review

In [5]:
# Review gene identifiers
# These identifiers appear to be standard human gene symbols
# Examples like ABCB1, B2M, ATM are all recognized human gene symbols

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols and prepare for linking
try:
    # Normalize gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    # Since we identified in Step 2 that there's no COVID-19 data available in this dataset
    # (trait_row is None), we cannot proceed with linking clinical and genetic data
    is_trait_available = False if 'trait_row' not in locals() else (trait_row is not None)
    
    # Validate data quality and save cohort info
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Set to True as we don't have trait data for analysis
        df=pd.DataFrame(),  # Empty DataFrame since we don't have linked data
        note="Gene expression data available but no COVID-19 trait data in this lung transplantation dataset."
    )
    
    print("Processing complete. Gene data saved but no clinical data linking possible due to missing trait information.")
    
except Exception as e:
    print(f"Error in data processing: {e}")
    
    # Log the error and mark the dataset as unusable
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=False,
        is_trait_available=False,
        is_biased=True,
        df=pd.DataFrame(),
        note=f"Error during normalization or processing: {str(e)}"
    )

Gene data shape after normalization: (576, 118)
Normalized gene data saved to ../../output/preprocess/COVID-19/gene_data/GSE273225.csv
Abnormality detected in the cohort: GSE273225. Preprocessing failed.
Processing complete. Gene data saved but no clinical data linking possible due to missing trait information.
